#1.Data_preparation

Готовим датасеты для обучения

Виды обработки текста: 
1) tokenization 2) stemming 3) lemmatization 4) stemming+misspelling 5) lemmatization+misspelling

Виды векторизации: 1) bag of words 2) word counts 3) TFIDF 4) word2vec(бонус)


In [2]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
!pip install pymorphy3
!pip install pattern
!pip install pyspellchecker

ModuleNotFoundError: No module named 'nltk'

In [ ]:
import pandas as pd
from pandas.core.frame import DataFrame
import numpy as np
import string
from nltk.stem import PorterStemmer,  WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
import re
import pymorphy3
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

## Читаем данные из файла
Собираем датафрейм, в котором колонка tweets содержит строки, колонка sentiments для классификации: 0 - негативный, 1 - нейтральный, 2 - позитивный

In [3]:
df = DataFrame(columns = ['Tweets', 'Sentiments'])

negative = pd.read_csv('p00_tweets/processedNegative.csv').T.reset_index()
neutral = pd.read_csv('p00_tweets/processedNeutral.csv').T.reset_index()
positive = pd.read_csv('p00_tweets/processedPositive.csv').T.reset_index()

negative['Sentiments'] = 2
neutral['Sentiments'] = 0
positive['Sentiments'] = 1

df = pd.concat([negative, neutral, positive]).reset_index(drop=True)
df.columns=['Tweets','Sentiments']
df


NameError: name 'DataFrame' is not defined

In [ ]:
# почистим слова от тегов, ссылок и небуквенных символов
def clean_text(text):
  text=re.sub(r'@[A-Za-z0-9]+','',text) # removed @mentions
  text=re.sub(r'#','',text) #removing symbol #
  text=re.sub(r'RT[\s]+','',text) #removing retweets
  text=re.sub(r'https?:\/\/\S+','',text) #removing hyperlinks
  text=re.sub(r'[^A-Za-z\s]','',text) #removing special characters
  text=text.lower()
  return text


## Токенизация и стоп-слова

Токенизация – разбиение строки на кусочки – токены (мы разбиваем по словам).

Stop Words - слова которые никак не влияются на содержание, типа “the, to, and, also”. Удаляем их.



In [ ]:
# этот токенайзер удаляет пунктуацию, включая апострофы
# после этого все стоп-слова норм убираются
stop_words = set(stopwords.words('english'))
tokenizer = RegexpTokenizer(r'\w+') 
def tokenize_text(text):
  res = []
  tokens = tokenizer.tokenize(text.lower())
  for w in tokens:
    if w not in stop_words:
      res.append(w)
  return clean_text(" ".join(res))

In [ ]:
df['Tokens']=df['Tweets'].apply(tokenize_text)
df


## Misspelling
Коррекция грамматических ошибок


In [ ]:
def misspelling(text):
  return(''.join(TextBlob(text).correct()))


In [ ]:
df['Spell'] = df['Tokens'].apply(misspelling)
df

## Стемминг
Оставляет у слова корень, обрезая окончания: making -> make, included -> includ

Стемминг быстрее лемматизации

In [ ]:
from nltk.corpus.reader import wordlist
def stem_text(words):
  res = []
  stemmer = PorterStemmer()
  for w in words.split():
    res.append(stemmer.stem(w))
  return(" ".join(res))

In [ ]:
df['Stem']=df['Tokens'].apply(stem_text)
df

In [ ]:
df['Stem_Spell']=df['Spell'].apply(stem_text)
df

## Лемматизация 
Возвращает лемму – базовую форму слова: better -> good, mice -> mouse.

Лемматизация точнее стемминга, но медленнее, потому что требует обращаться к словарю

In [ ]:
#ntlk WordNetLemmatizer - хорошо работает если разметить типы слова (глагол, сущ, прилагательное)
def get_wordnet_pos(tag):
  from nltk.corpus.reader.wordnet import VERB, NOUN, ADJ, ADV
  if tag.startswith('J'):
      return ADJ
  elif tag.startswith('V'):
      return VERB
  elif tag.startswith('N'):
      return NOUN
  elif tag.startswith('R'):
      return ADV
  else:
      return NOUN

def lemmatize_text(words):
  res = []
  lemmatizer = WordNetLemmatizer()
  tagged = nltk.pos_tag(words.split()) # расставляет теги в словах для лучшей лемматизации
  for w, tag in tagged:
    res.append(lemmatizer.lemmatize(w, get_wordnet_pos(tag)))
  return(' '.join(res))

In [ ]:
df['Lemm']=df['Tokens'].apply(lemmatize_text)
df

In [ ]:
df['Lemm_Spell']=df['Spell'].apply(lemmatize_text)
df

In [ ]:
#сохраним датафрейм в файл
df.to_csv('/content/drive/MyDrive/p00_tweets/data/tweets.csv')

##Векторизация

Переведем все предложения в набор цифр, чтобы на этих наборах обучать модель. 

In [ ]:
tweet1 = "unhappi dog like though one"
tweet2 = "miss go gig liverpool unhappi one"
tweet3 = "isnt one new riverdal tonight unhappi"
text = [tweet1, tweet2, tweet3]
text

In [ ]:
binary_vect = CountVectorizer(binary=True)
arr1 = binary_vect.fit_transform(text).toarray()
arr1


In [ ]:
count_vect = CountVectorizer(max_df=0.75)
arr2 = count_vect.fit_transform(text).toarray()
arr2

In [ ]:
tdidf_vect = TfidfVectorizer()
arr3 = tdidf_vect.fit_transform(text).toarray()
np.set_printoptions(precision=2)
print(arr3)

Параметры CountVectorizer():
- binary=True - не принимается во внимание частота слова. Если слово встретилось - 1, если нет - 0
- max_features - ограничение по максимальному количеству слов в словаре
- max_df ограничивает максимальную частоту встречющегося слова - если слово слишком часто встречается, оно не несет информации, например "he". max_df=0.75 убирает из словаря слова, встречающиеся более чем в 75% твитов.
- min_df ограничивает минимальную частоту встречаемого слова - если слово слишком редкое, то его можно игнорировать, например имена собственные. min_df=5 убирает из словаря слова, встречающиеся менее чем в 5 твитах.
- ngram_range учитывает повторяющиеся последовательности слов в заданном диапазоне

## 0 or 1
Создаем словарь из всех слов, которые есть в предложениях. Каждое предложение представляем в виде набора 0 и 1. 1 - если слово присутствует, 0 - остутствует

In [ ]:
def vectorize_01(col):
  vectorizer = CountVectorizer(binary=True)
  X = vectorizer.fit_transform(col).toarray()
  # print(X.shape)
  # print(X)
  return X

In [ ]:
X01=vectorize_01(df['Tokens'])
X02=vectorize_01(df['Stem'])
X03=vectorize_01(df['Lemm'])
X04=vectorize_01(df['Stem_Spell'])
X05=vectorize_01(df['Lemm_Spell'])


##Count words
Учитывает частоту, с которой слово встречается в словаре. Благодяря этому можно отсеять самы частые и самые редкие слова и ограничить словарь

In [ ]:
def vectorize_word_counts(col):
  vectorizer = CountVectorizer(max_features=3000, min_df=5, max_df=0.7, ngram_range=(2, 4), analyzer='char_wb') 
  X = vectorizer.fit_transform(col).toarray()
  # print(X.shape)
  # print(X)
  return X

In [ ]:
X06=vectorize_word_counts(df['Tokens'])
X07=vectorize_word_counts(df['Stem'])
X08=vectorize_word_counts(df['Lemm'])
X09=vectorize_word_counts(df['Stem_Spell'])
X10=vectorize_word_counts(df['Lemm_Spell'])

##TFIDF
Считает отношение частоты встречаемости слов к общему числу слов в документе. Таким образом, каждому слову присваивается "вес" который будет учитываться при обучении модели

In [ ]:
def vectorize_tdidf(col):
  tfidfconverter = TfidfVectorizer(ngram_range=(1,2))
  X = tfidfconverter.fit_transform(col).toarray()
  # print(X.shape)
  # print(X)
  return X

In [ ]:
X11=vectorize_tdidf(df['Tokens'])
X12=vectorize_tdidf(df['Stem'])
X13=vectorize_tdidf(df['Lemm'])
X14=vectorize_tdidf(df['Stem_Spell'])
X15=vectorize_tdidf(df['Lemm_Spell'])

##Train test split

In [ ]:
y = df['Sentiments']

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
y_train, y_test = train_test_split(y, test_size=0.2, stratify=df['Sentiments'], random_state=42)
X01_train, X01_test = train_test_split(X01, test_size=0.2, stratify=df['Sentiments'], random_state=42) # добавить стратификацию
X02_train, X02_test = train_test_split(X02, test_size=0.2, random_state=42)
X03_train, X03_test = train_test_split(X03, test_size=0.2, random_state=42)
X04_train, X04_test = train_test_split(X04, test_size=0.2, random_state=42)
X05_train, X05_test = train_test_split(X05, test_size=0.2, random_state=42)
X06_train, X06_test = train_test_split(X06, test_size=0.2, random_state=42)
X07_train, X07_test = train_test_split(X07, test_size=0.2, random_state=42)
X08_train, X08_test = train_test_split(X08, test_size=0.2, random_state=42)
X09_train, X09_test = train_test_split(X09, test_size=0.2, random_state=42)
X10_train, X10_test = train_test_split(X10, test_size=0.2, random_state=42)
X11_train, X11_test = train_test_split(X11, test_size=0.2, random_state=42)
X12_train, X12_test = train_test_split(X12, test_size=0.2, random_state=42)
X13_train, X13_test = train_test_split(X13, test_size=0.2, random_state=42)
X14_train, X14_test = train_test_split(X14, test_size=0.2, random_state=42)
X15_train, X15_test = train_test_split(X15, test_size=0.2, random_state=42)

print(y_train.shape)
print(y_test.shape)


##Random forest

In [ ]:
results = DataFrame(index = ['tokenization', 'stemming', 'lemmatization', 'stemming+misspelling', 'lemmatization+misspelling'], 
                    columns=['0or1', 'word_counts', 'TFIDF', 'word2vec'])


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

classifier = RandomForestClassifier(n_estimators=200, random_state=0)

In [ ]:
classifier.fit(X01_train, y_train)
y_pred = classifier.predict(X01_test)

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))
accuracy01 = accuracy_score(y_test, y_pred)
results.loc['tokenization', '0or1'] = accuracy01

In [ ]:
classifier.fit(X02_train, y_train)
y_pred = classifier.predict(X02_test)

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))
accuracy02 = accuracy_score(y_test, y_pred)
results.loc['stemming', '0or1'] = accuracy02

In [ ]:
classifier.fit(X03_train, y_train)
y_pred = classifier.predict(X03_test)

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))
accuracy03 = accuracy_score(y_test, y_pred)
results.loc['lemmatization', '0or1'] = accuracy03

In [ ]:
classifier.fit(X04_train, y_train)
y_pred = classifier.predict(X04_test)

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))
accuracy04 = accuracy_score(y_test, y_pred)
results.loc['stemming+misspelling', '0or1'] = accuracy04

In [ ]:
classifier.fit(X05_train, y_train)
y_pred = classifier.predict(X05_test)

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))
accuracy05 = accuracy_score(y_test, y_pred)
results.loc['lemmatization+misspelling', '0or1'] = accuracy05

In [ ]:
classifier.fit(X06_train, y_train)
y_pred = classifier.predict(X06_test)

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))
accuracy06 = accuracy_score(y_test, y_pred)
results.loc['tokenization', 'word_counts'] = accuracy06

In [ ]:
classifier.fit(X07_train, y_train)
y_pred = classifier.predict(X07_test)

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))
accuracy07 = accuracy_score(y_test, y_pred)
results.loc['stemming', 'word_counts'] = accuracy07

In [ ]:
classifier.fit(X08_train, y_train)
y_pred = classifier.predict(X08_test)

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))
accuracy08 = accuracy_score(y_test, y_pred)
results.loc['lemmatization', 'word_counts'] = accuracy08

In [ ]:
classifier.fit(X09_train, y_train)
y_pred = classifier.predict(X09_test)

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))
accuracy09 = accuracy_score(y_test, y_pred)
results.loc['stemming+misspelling', 'word_counts'] = accuracy09

In [ ]:
classifier.fit(X10_train, y_train)
y_pred = classifier.predict(X10_test)

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))
accuracy10 = accuracy_score(y_test, y_pred)
results.loc['lemmatization+misspelling', 'word_counts'] = accuracy10

In [ ]:
classifier.fit(X11_train, y_train)
y_pred = classifier.predict(X11_test)

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))
accuracy11 = accuracy_score(y_test, y_pred)
results.loc['tokenization', 'TFIDF'] = accuracy11

In [ ]:
classifier.fit(X12_train, y_train)
y_pred = classifier.predict(X12_test)

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))
accuracy12 = accuracy_score(y_test, y_pred)
results.loc['stemming', 'TFIDF'] = accuracy12

In [ ]:
classifier.fit(X13_train, y_train)
y_pred = classifier.predict(X13_test)

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))
accuracy13 = accuracy_score(y_test, y_pred)
results.loc['lemmatization', 'TFIDF'] = accuracy13

In [ ]:
classifier.fit(X14_train, y_train)
y_pred = classifier.predict(X14_test)

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))
accuracy14 = accuracy_score(y_test, y_pred)
results.loc['stemming+misspelling', 'TFIDF'] = accuracy14

In [ ]:
classifier.fit(X15_train, y_train)
y_pred = classifier.predict(X15_test)

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))
accuracy15 = accuracy_score(y_test, y_pred)
results.loc['lemmatization+misspelling', 'TFIDF'] = accuracy15

In [ ]:

results

План
- ✔ (работает, но оооооч медленно) misspelling - Ира
- ✅ разобраться с параметрами в векторизации ф-ция CountVectorizer - Ира - вот здесь хорошо описано: https://itchef.ru/articles/51879/
- попробовать сохранить массив в файл и прочитать его по-нормальному - Юля
- сделать ф-цию которая будет делать сплит трейн тест - Ира
- штука которая сравнивает результаты - Юля
- подумать как все датасеты загнать в модель - Юля, Ира
- посмотреть другие модели - Юля
- vord to vec - Юля, Ира
- 10 наиболее совпадающих твитов в каждом датасете (массив numpy) - Юля, Ира